In [1]:
!pip install googledrivedownloader
!pip install keras

    100% |████████████████████████████████| 327kB 2.1MB/s 


In [2]:
from google_drive_downloader import GoogleDriveDownloader as gdd

ids =["1o4FNPEsiJyTAkexCzR1P85WgaODJZTMs","1yiP1nS5bCKHYto0Px3f_BOUbyJJVdKRO",
"1f3wDVtDq1oedLJ0Hidn2gduttRacAtoD", "1Svv2H8HgF-gyE3GgHJ4Kgcpb7_l88tBl",
"1aJQwc58seuB4tyIV3tZCFQBbSsI0CAVJ", "1Q77JWfZLXc_BmFoj0rQoIojvtloTkk2f",
"1bYFaYWfmmAgfj4hyEQXSsY_PrMt9YF0l", "1ZKhkmCHBFrrocGhmAG55Wgkd-ewZmd9l",
"1Z7FiOG8iTfUBnogZw8SQOPd1NDjQ79CO"]

file_names = ["bed.pkl","bird.pkl", "cat.pkl", "dog.pkl", "down.pkl", "eight.pkl", "five.pkl", "four.pkl", "go.pkl"]
for i in range(len(ids)):
  a = gdd.download_file_from_google_drive(file_id= ids[i],
                                      dest_path="./"+file_names[i],
                                      unzip=False)

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten, BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import backend as K
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
K.set_image_dim_ordering('th')
import pickle

Using TensorFlow backend.


In [0]:
"""words list represents the "pkl" file names
    which have the same word as .wav file.
    The pkl file is dictionary with key as the word spoken
    and the value as numpy array where element is a (20 * 100) vector
    representing the mfcc vectors of the .wav file. """

words = ["bed", "down", "dog", "eight", "four", "cat","go","bird","five"]
#words = ["bed", "down", "dog"]

In [5]:
data = pickle.load( open( words[0]+".pkl", "rb" ) )
for keys in data:
    X = data[keys]
    Y = np.array([[keys]] * data[keys].shape[0])
    #print X_1.shape
    
    
for i in range(1,len(words)):
    data = pickle.load( open( words[i]+".pkl", "rb" ) )
    for keys in data:
        Y = np.append(Y, [[keys]] * data[keys].shape[0], axis = 0)
        X = np.append(X, data[keys], axis = 0)
        #print X_1.shape
        
print X.shape
print Y.shape
print Y

(18735, 20, 100)
(18735, 1)
[['bed']
 ['bed']
 ['bed']
 ...
 ['five']
 ['five']
 ['five']]


In [6]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(Y)
#print integer_encoded
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
Y = onehot_encoded

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
"""Convolution Layer in Keras
   accepts input as 3d - height, width, depth(RGB channels).
   Since we do not have depth in text, we can convert it to a vector
   with a depth = 1"""

X = X.reshape(X.shape[0], 1, 20, 100)
print X.shape
print Y.shape
num_classes = Y.shape[1]
print num_classes

(18735, 1, 20, 100)
(18735, 9)
9


In [0]:
model = Sequential()
model.add(Conv2D(50, (5, 5), input_shape=(1, 20, 100), activation='relu', padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(30, (3, 3), activation='relu', padding="same"))
model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(15, (3, 3), activation='relu', padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 20, 100)       1300      
_________________________________________________________________
batch_normalization_1 (Batch (None, 50, 20, 100)       400       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 10, 50)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 10, 50)        13530     
_________________________________________________________________
batch_normalization_2 (Batch (None, 30, 10, 50)        200       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 10, 50)        4065      
_________________________________________________________________
batch_normalization_3 (Batch (None, 15, 10, 50)        200       
__________

In [10]:
file_path="CNN_v4_val_loss.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)
callbacks_list = [checkpoint, early] #early
model.fit(X, Y, validation_split=0.1, callbacks=callbacks_list, epochs=1, batch_size= 32)

Train on 16861 samples, validate on 1874 samples
Epoch 1/1
16861/16861 [==============================] - 13s 753us/step - loss: 1.5056 - acc: 0.4504 - val_loss: 3.2292 - val_acc: 0.0390

Epoch 00001: val_loss improved from inf to 3.22918, saving model to CNN_v4_val_loss.hdf5


In [0]:
model = load_model(file_path)

In [12]:
model.fit(X, Y, validation_split=0.1, callbacks=callbacks_list, epochs=20, batch_size= 32)

Train on 16861 samples, validate on 1874 samples
Epoch 1/20
16861/16861 [==============================] - 12s 708us/step - loss: 0.7257 - acc: 0.7547 - val_loss: 1.2424 - val_acc: 0.6110

Epoch 00001: val_loss improved from 3.22918 to 1.24239, saving model to CNN_v4_val_loss.hdf5
Epoch 2/20
16861/16861 [==============================] - 11s 625us/step - loss: 0.5054 - acc: 0.8310 - val_loss: 0.6115 - val_acc: 0.8212

Epoch 00002: val_loss improved from 1.24239 to 0.61145, saving model to CNN_v4_val_loss.hdf5
Epoch 3/20
 1088/16861 [>.............................] - ETA: 9s - loss: 0.4170 - acc: 0.8557

16861/16861 [==============================] - 10s 620us/step - loss: 0.4033 - acc: 0.8644 - val_loss: 0.6235 - val_acc: 0.8074

Epoch 00003: val_loss did not improve
Epoch 4/20
16861/16861 [==============================] - 10s 615us/step - loss: 0.3419 - acc: 0.8851 - val_loss: 0.8383 - val_acc: 0.7417

Epoch 00004: val_loss did not improve
Epoch 5/20
 8672/16861 [==============>...............] - ETA: 4s - loss: 0.2994 - acc: 0.8990

16861/16861 [==============================] - 10s 618us/step - loss: 0.2992 - acc: 0.8994 - val_loss: 0.8107 - val_acc: 0.7476

Epoch 00005: val_loss did not improve
Epoch 6/20
16861/16861 [==============================] - 10s 610us/step - loss: 0.2721 - acc: 0.9091 - val_loss: 0.4074 - val_acc: 0.8757

Epoch 00006: val_loss improved from 0.61145 to 0.40737, saving model to CNN_v4_val_loss.hdf5
Epoch 7/20
 8960/16861 [==============>...............] - ETA: 4s - loss: 0.2317 - acc: 0.9213

16861/16861 [==============================] - 10s 612us/step - loss: 0.2469 - acc: 0.9172 - val_loss: 0.4242 - val_acc: 0.8655

Epoch 00007: val_loss did not improve
Epoch 8/20
16861/16861 [==============================] - 10s 618us/step - loss: 0.2216 - acc: 0.9242 - val_loss: 0.4416 - val_acc: 0.8714

Epoch 00008: val_loss did not improve
Epoch 9/20
10112/16861 [================>.............] - ETA: 4s - loss: 0.2085 - acc: 0.9311

16861/16861 [==============================] - 10s 619us/step - loss: 0.2123 - acc: 0.9294 - val_loss: 0.4232 - val_acc: 0.8698

Epoch 00009: val_loss did not improve
Epoch 10/20
16861/16861 [==============================] - 10s 619us/step - loss: 0.1936 - acc: 0.9361 - val_loss: 0.4005 - val_acc: 0.8858

Epoch 00010: val_loss improved from 0.40737 to 0.40049, saving model to CNN_v4_val_loss.hdf5
Epoch 11/20
 9056/16861 [===============>..............] - ETA: 4s - loss: 0.1782 - acc: 0.9387

16861/16861 [==============================] - 10s 612us/step - loss: 0.1818 - acc: 0.9390 - val_loss: 0.6741 - val_acc: 0.8159

Epoch 00011: val_loss did not improve
Epoch 12/20
16861/16861 [==============================] - 10s 611us/step - loss: 0.1796 - acc: 0.9374 - val_loss: 0.3659 - val_acc: 0.8949

Epoch 00012: val_loss improved from 0.40049 to 0.36592, saving model to CNN_v4_val_loss.hdf5
Epoch 13/20
 8960/16861 [==============>...............] - ETA: 4s - loss: 0.1619 - acc: 0.9449

16861/16861 [==============================] - 10s 614us/step - loss: 0.1628 - acc: 0.9441 - val_loss: 0.3850 - val_acc: 0.8831

Epoch 00013: val_loss did not improve
Epoch 14/20
16861/16861 [==============================] - 10s 617us/step - loss: 0.1509 - acc: 0.9482 - val_loss: 0.4375 - val_acc: 0.8858

Epoch 00014: val_loss did not improve
Epoch 15/20
 9952/16861 [================>.............] - ETA: 4s - loss: 0.1407 - acc: 0.9522

16861/16861 [==============================] - 10s 619us/step - loss: 0.1483 - acc: 0.9502 - val_loss: 0.7604 - val_acc: 0.7930

Epoch 00015: val_loss did not improve
Epoch 16/20
16861/16861 [==============================] - 10s 614us/step - loss: 0.1414 - acc: 0.9505 - val_loss: 0.2888 - val_acc: 0.9216

Epoch 00016: val_loss improved from 0.36592 to 0.28875, saving model to CNN_v4_val_loss.hdf5
Epoch 17/20
 9056/16861 [===============>..............] - ETA: 4s - loss: 0.1283 - acc: 0.9561

16861/16861 [==============================] - 10s 617us/step - loss: 0.1279 - acc: 0.9559 - val_loss: 0.4618 - val_acc: 0.8703

Epoch 00017: val_loss did not improve
Epoch 18/20
16861/16861 [==============================] - 10s 618us/step - loss: 0.1255 - acc: 0.9571 - val_loss: 0.2920 - val_acc: 0.9242

Epoch 00018: val_loss did not improve
Epoch 19/20
10016/16861 [================>.............] - ETA: 4s - loss: 0.1170 - acc: 0.9604

16861/16861 [==============================] - 10s 616us/step - loss: 0.1279 - acc: 0.9576 - val_loss: 0.4663 - val_acc: 0.8741

Epoch 00019: val_loss did not improve
Epoch 20/20
16861/16861 [==============================] - 10s 616us/step - loss: 0.1088 - acc: 0.9622 - val_loss: 0.3402 - val_acc: 0.9120

Epoch 00020: val_loss did not improve


In [0]:
model = load_model(file_path)

In [0]:
predicted = model.predict(X)

In [0]:
from sklearn import metrics

In [16]:
metrics.log_loss(Y, predicted)

0.06770054147197994